In [9]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [10]:
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [11]:
# Opening the website and declaring the variable
# Path driver setup is done in command using export PATH
driver = webdriver.Chrome()

web = 'https://www.amazon.fr/'

options = webdriver.ChromeOptions()
options.add_argument('--headless')

# create a driver object using driver_path as a parameter
driver.get(web)

In [12]:
# assign any keyword for searching
keyword = "raspberry pi 4"
# create WebElement for a search box
search_box = driver.find_element_by_id('twotabsearchtextbox')
# type the keyword in searchbox
search_box.send_keys(keyword)
# create WebElement for a search button
search_button = driver.find_element_by_id('nav-search-submit-button')
# click search_button
search_button.click()
# wait for the page to download
driver.implicitly_wait(5)
# quit the driver after finishing scraping (please keep this line at the bottom)
#driver.quit()

/tmp/ipykernel_57036/3835606142.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = driver.find_element_by_id('twotabsearchtextbox')
/tmp/ipykernel_57036/3835606142.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_id('nav-search-submit-button')


##### On veut créer une liste des produits en Amazon pour montrer les prix des raspberry pi , initialization des cases pour chaque produits

In [13]:
product_name = []
product_asin = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []

In [14]:
items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))


In [15]:
for item in items:
    name = item.find_element(By.XPATH,'.//span [@class="a-size-base-plus a-color-base a-text-normal"]')
    product_name.append(name.text)

    data_asin = item.get_attribute("data-asin")
    product_asin.append(data_asin)
    # find prices
    whole_price = item.find_elements(By.XPATH,'.//span[@class="a-price-whole"]')
    #fraction_price = item.find_elements(By.XPATH,'.//span[@class="a-price-fraction"]')
    #if whole_price != [] and fraction_price != []:
    #    price = '.'.join([whole_price[0].text, fraction_price[0].text])
    if whole_price != []:
        price = '.'.join([whole_price[0].text])
    else:
        price = 0
    product_price.append(price)

    # find a ratings box
    ratings_box = item.find_elements_by_xpath('.//div[@class="a-row a-size-small"]/span')
    if ratings_box != []:
        ratings = ratings_box[0].get_attribute('aria-label')
        ratings_num = ratings_box[1].get_attribute('aria-label')
    else:
        ratings, ratings_num = 0, 0
    product_ratings.append(ratings)
    product_ratings_num.append(str(ratings_num))

    link = item.find_element_by_xpath('.//a[@class="a-link-normal a-text-normal"]').get_attribute("href")
    product_link.append(link)

# to check data scraped
print(product_name)
print(product_asin)
print(product_price)
print(product_ratings)
print(product_ratings_num)
print(product_link)

/home/solomoon/Workspace/M2-IOT/projet-scraping/env/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/home/solomoon/Workspace/M2-IOT/projet-scraping/env/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


['Raspberry Pi 4 Modèle B 4Go avec 32Go RAM Class10 Carte MicroSD, Prend en Charge le Double Affichage 4K/1000Mbps/Bluetooth 5.0, 5V 3A USB-C ON/OFF Alimentation à Découpage et Câble Micro HDMI -RPi 432', 'Vemico Raspberry Pi 4 Modèle B Starter Kit 4G RAM + 32GB Carte Micro SD/Câble Micro HDMI/Boîtier ABS/Lecteur de Carte/USB Type C Adaptateur Secteur/Ventilateur/Dissipateur de Chaleur', '@WENDi Raspberry Pi 4 Model B Starter Kit, Micro SD Card, Heat Sink, Power Supply', 'Flirc Boîtier Raspberry Pi 4 (édition Kodi)', 'Raspberry Pi 4 Modèle B 4 Go ARM-Cortex-A72 4 x 1,50 GHz, 4 Go de RAM, WLAN-AC, Bluetooth 5, LAN, 4 x USB, 2 x Micro HDMI', 'Raspberry Pi 4 Modèle B 4Go avec 32Go RAM Class10 Carte MicroSD, Prend en Charge le Double Affichage 4K/1000Mbps/Bluetooth 5.0, 5V 3A USB-C ON/OFF Alimentation à Découpage et Câble Micro HDMI -RPi 432', 'Raspberry Pi 4 Modèle B 4Go avec 64Go RAM Class10 Carte MicroSD, Prend en Charge le Double Affichage 4K/1000Mbps/Bluetooth 5.0, 5V 3A USB-C ON/OFF 

##### Storage de data en liste


In [52]:
import sqlite3

def store_db(product_asin, product_name, product_price, product_ratings, product_ratings_num, product_link):
    conn = sqlite3.connect('amazon_search.db')
    curr = conn.cursor()

    # create table
    
    curr.execute('''CREATE TABLE IF NOT EXISTS search_result (ASIN text, name text, price real, ratings text, ratings_num text, details_link text)''')
    # insert data into a table
    curr.executemany("INSERT INTO search_result (ASIN, name, price, ratings, ratings_num, details_link) VALUES (?,?,?,?,?,?)", 
                    list(zip(product_asin, product_name, product_price, product_ratings, product_ratings_num, product_link)))
        
    conn.commit()
    conn.close()

In [53]:

options = webdriver.ChromeOptions()
options.add_argument('--headless')

keyword = 'raspberry pi 4'
next_page = ''

In [54]:
def scrape_amazon(keyword, max_pages):

    page_number = 1
    driver = webdriver.Chrome()
    web = 'https://www.amazon.fr/'

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')

    # create a driver object using driver_path as a parameter
    driver.get(web)

    driver.implicitly_wait(5)
    keyword = keyword
    search_box = driver.find_element_by_id('twotabsearchtextbox')
    search_box.send_keys(keyword)
    search_button = driver.find_element_by_id('nav-search-submit-button')
    search_button.click()
    
    driver.implicitly_wait(5)

    while page_number <= max_pages:
        scrape_page(driver)
        driver.get(next_page)
        driver.implicitly_wait(5)
        page_number += 1
    driver.quit()

In [55]:
def scrape_page(driver):
    
    product_name = []
    product_asin = []
    product_price = []
    product_ratings = []
    product_ratings_num = []
    product_link = []
    
    items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
    for item in items:
        name = item.find_element(By.XPATH,'.//span [@class="a-size-base-plus a-color-base a-text-normal"]')
        product_name.append(name.text)

        data_asin = item.get_attribute("data-asin")
        product_asin.append(data_asin)
        # find prices
        whole_price = item.find_elements(By.XPATH,'.//span[@class="a-price-whole"]')
        #fraction_price = item.find_elements(By.XPATH,'.//span[@class="a-price-fraction"]')
        #if whole_price != [] and fraction_price != []:
        #    price = '.'.join([whole_price[0].text, fraction_price[0].text])
        if whole_price != []:
            price = '.'.join([whole_price[0].text])
        else:
            price = 0
        product_price.append(price)

        # find a ratings box
        ratings_box = item.find_elements_by_xpath('.//div[@class="a-row a-size-small"]/span')
        if ratings_box != []:
            ratings = ratings_box[0].get_attribute('aria-label')
            ratings_num = ratings_box[1].get_attribute('aria-label')
        else:
            ratings, ratings_num = 0, 0
        product_ratings.append(ratings)
        product_ratings_num.append(str(ratings_num))

        link = item.find_element_by_xpath('.//a[@class="a-link-normal a-text-normal"]').get_attribute("href")
        product_link.append(link)
    # store data from lists to database
    global next_page
    next_page = driver.find_element(By.XPATH,'//li[@class ="a-selected"]/following-sibling::li/a').get_attribute("href")
    
    store_db(product_asin, product_name, product_price, product_ratings, product_ratings_num, product_link)


In [56]:
scrape_amazon(keyword,2)


/tmp/ipykernel_57036/497490757.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = driver.find_element_by_id('twotabsearchtextbox')
/tmp/ipykernel_57036/497490757.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_id('nav-search-submit-button')
/home/solomoon/Workspace/M2-IOT/projet-scraping/env/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/home/solomoon/Workspace/M2-IOT/projet-scraping/env/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Plea

In [58]:
print()

['4,7 sur 5\xa0étoiles', '5,0 sur 5\xa0étoiles', 0, '4,7 sur 5\xa0étoiles', '4,7 sur 5\xa0étoiles', '4,7 sur 5\xa0étoiles', '4,5 sur 5\xa0étoiles', 0, '4,6 sur 5\xa0étoiles', '4,7 sur 5\xa0étoiles', 0, '3,3 sur 5\xa0étoiles', '4,5 sur 5\xa0étoiles', '5,0 sur 5\xa0étoiles', '4,5 sur 5\xa0étoiles', '4,6 sur 5\xa0étoiles', '4,5 sur 5\xa0étoiles', '4,8 sur 5\xa0étoiles', 0, '4,7 sur 5\xa0étoiles', '4,4 sur 5\xa0étoiles', '4,0 sur 5\xa0étoiles', '4,3 sur 5\xa0étoiles', '4,6 sur 5\xa0étoiles', '4,5 sur 5\xa0étoiles', '5,0 sur 5\xa0étoiles', '4,2 sur 5\xa0étoiles', '4,3 sur 5\xa0étoiles', '4,6 sur 5\xa0étoiles', '5,0 sur 5\xa0étoiles', '4,6 sur 5\xa0étoiles', '3,7 sur 5\xa0étoiles', '4,7 sur 5\xa0étoiles', '4,6 sur 5\xa0étoiles', '4,6 sur 5\xa0étoiles', 0, '4,3 sur 5\xa0étoiles', '4,5 sur 5\xa0étoiles', '4,4 sur 5\xa0étoiles', 0, '4,5 sur 5\xa0étoiles', '4,3 sur 5\xa0étoiles', '4,2 sur 5\xa0étoiles', '3,8 sur 5\xa0étoiles', '4,6 sur 5\xa0étoiles', '4,8 sur 5\xa0étoiles', '4,7 sur 5\xa0étoiles